# Bank Loan Default - BLD
[Deloitte Hackaton](https://www.kaggle.com/datasets/ankitkalauni/bank-loan-defaulter-prediction-hackathon/data?select=train.csv)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [2]:
#   Import datasets
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [10]:
#   Check if columns are the same between datasets
print('Are the columns between train and test the same?', '\n',test.columns.equals(train.columns))
test.columns

Are the columns between train and test the same? 
 True


Index(['ID', 'Loan Amount', 'Funded Amount', 'Funded Amount Investor', 'Term',
       'Batch Enrolled', 'Interest Rate', 'Grade', 'Sub Grade',
       'Employment Duration', 'Home Ownership', 'Verification Status',
       'Payment Plan', 'Loan Title', 'Debit to Income',
       'Delinquency - two years', 'Inquires - six months', 'Open Account',
       'Public Record', 'Revolving Balance', 'Revolving Utilities',
       'Total Accounts', 'Initial List Status', 'Total Received Interest',
       'Total Received Late Fee', 'Recoveries', 'Collection Recovery Fee',
       'Collection 12 months Medical', 'Application Type', 'Last week Pay',
       'Accounts Delinquent', 'Total Collection Amount',
       'Total Current Balance', 'Total Revolving Credit Limit', 'Loan Status'],
      dtype='object')

In [13]:
#   Cehck dataset head
train.head(2)

,ID,Loan Amount,Funded Amount,Funded Amount Investor,Term,Batch Enrolled,Interest Rate,Grade,Sub Grade,Employment Duration,...,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Application Type,Last week Pay,Accounts Delinquent,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status
0,65087372,10000,32236,12329.36286,59,BAT2522922,11.135007,B,C4,MORTGAGE,...,2.498291,0.793724,0,INDIVIDUAL,49,0,31,311301,6619,0
1,1450153,3609,11940,12191.99692,59,BAT1586599,12.237563,C,D3,RENT,...,2.377215,0.974821,0,INDIVIDUAL,109,0,53,182610,20885,0


According to the dataset documentation, the variables are the following:

0. ID (Int) ---> unique ID of representative
1. Loan Amount (Int) ---> loan amount applied
2. Funded Amount (Int) ---> loan amount funded
3. Funded Amount Investor (Float) ---> loan amount approved by the investors
4. Term (Int) ---> term of loan (in months)
5. Batch Enrolled (Object) ---> batch numbers to representatives
6. Interest Rate (Float) ---> interest rate (%) on loan
7. Grade (Object) ---> grade by the bank
8. Sub Grade (Object) ---> sub-grade by the bank
9. Employment Duration (Object) ---> duration
10. Home Ownership (Float) ---> Ownership of home
11. Verification Status (Object) ---> Income verification by the bank
12. Payment Plan (Object) ---> if any payment plan has started against loan
13. Loan Title (Object) ---> loan title provided
14. Debit to Income (Float) ---> ratio of representative's total monthly debt repayment
15. Delinquency - two years (Int) ---> number of 30+ days delinquency in past 2 years
16. Inquires - six months (Int) ---> total number of inquiries in last 6 months
17. Open Account (Int) ---> number of open credit line in representative's credit line
18. Public Record (Int) ---> number of derogatory public records
19. Revolving Balance (Int) ---> total credit revolving balance
20. Revolving Utilities (Float) ---> amount of credit a representative is using relative to revolving_balance
21. Total Accounts (Int) ---> total number of credit lines available in representatives credit line
22. Initial List Status (Object) ---> unique listing status of the loan - W(Waiting), F(Forwarded)
23. Total Received Interest (Float) ---> total interest received till date
24. Total Received Late Fee (Float) ---> total late fee received till date
25. Recoveries (Float) ---> post charge off gross recovery
26. Collection Recovery Fee (Float) ---> post charge off collection fee
27. Collection 12 months Medical (Int) ---> total collections in last 12 months excluding medical collections
28. Application Type (Object) ---> indicates when the representative is an individual or joint
29. Last week Pay (Int) ---> indicates how long (in weeks) a representative has paid EMI after batch enrolled
30. Accounts Delinquent (Int) ---> number of accounts on which the representative is delinquent
31. Total Collection Amount (Int) ---> total current balance from all accounts
32. Total Current Balance (Int) ---> total current balance from all accounts
33. Total Revolving Credit Limit (Int) ---> total revolving credit limit
34. Loan Status (Int) ---> 1 = Defaulter, 0 = Non Defaulters (Target feature)

## Avarages

In [18]:
#   Mean value and other statistical data for numerical data
round(train.describe(exclude=[object]),2)

,ID,Loan Amount,Funded Amount,Funded Amount Investor,Term,Interest Rate,Home Ownership,Debit to Income,Delinquency - two years,Inquires - six months,...,Total Received Late Fee,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Last week Pay,Accounts Delinquent,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status
count,67463.00,67463.00,67463.00,67463.00,67463.00,67463.00,67463.00,67463.00,67463.00,67463.00,...,67463.00,67463.00,67463.00,67463.00,67463.00,67463.0,67463.00,67463.00,67463.00,67463.00
mean,25627607.75,16848.90,15770.60,14621.80,58.17,11.85,80541.50,23.30,0.33,0.15,...,1.14,59.69,1.13,0.02,71.16,0.0,146.47,159573.93,23123.01,0.09
std,21091554.02,8367.87,8150.99,6785.35,3.33,3.72,45029.12,8.45,0.80,0.47,...,5.24,357.03,3.49,0.14,43.32,0.0,744.38,139033.25,20916.70,0.29
min,1297933.00,1014.00,1014.00,1114.59,36.00,5.32,14573.54,0.68,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,1.00,617.00,1000.00,0.00
25%,6570288.00,10012.00,9266.50,9831.68,58.00,9.30,51689.84,16.76,0.00,0.00,...,0.02,1.63,0.48,0.00,35.00,0.0,24.00,50379.00,8155.50,0.00
50%,17915646.00,16073.00,13042.00,12793.68,59.00,11.38,69335.83,22.66,0.00,0.00,...,0.04,3.34,0.78,0.00,68.00,0.0,36.00,118369.00,16733.00,0.00
75%,42715208.50,22106.00,21793.00,17807.59,59.00,14.19,94623.32,30.05,0.00,0.00,...,0.07,5.45,1.07,0.00,105.00,0.0,46.00,228375.00,32146.50,0.00
max,72245779.00,35000.00,34999.00,34999.75,59.00,27.18,406561.54,39.63,8.00,5.00,...,42.62,4354.47,166.83,1.00,161.00,0.0,16421.00,1177412.00,201169.00,1.00


Checking the statistics on the numerical features. We get an insight on the loans situation.
- The avarage loan amount is 16848.90 rupees (avg $198.08 in 2024 rate)
- Clients are expected to pay in 59 months (about 5 years), which seems to be the max term payment the bank offers
- The avarage interest rate in these loans is 11.85%
- Based on the bank data, loan clients have de compromised aroun 23.30% of their reported income to repay their loans
- The bank doesn't seems to have many late payments, the amount recovered after late payments is avg 59.69 rupees; However, we see an outlier with 4354.47 rupees in late payment, this must be pulling the avarage up. When we look the percentiles all them are under 6 rupees. We'll need to address outliers matter in this project as well.

## Loan Grades

In [24]:
#   Sub df for grades
train_grades = train.groupby('Grade')
test_grades = test.groupby('Grade')

#   Sub df for subrgades
train_subgrades = train.groupby('Sub Grade')
test_subgrades = test.groupby('Sub Grade')